In [206]:
import cv2
import numpy as np

In [207]:
translation_matches = [[(371, 349), (471, 549)],
 [(467, 187), (567, 387)],
 [(644, 182), (744, 382)],
 [(625, 790), (725, 990)],
 [(456, 450), (556, 650)],
 [(256, 445), (356, 645)],
 [(508, 509), (608, 709)],
 [(300, 83), (400, 283)],
 [(325, 573), (425, 773)],
 [(449, 339), (549, 539)]]

translation_matches, translation_matches_test = translation_matches[0], translation_matches[1:]

euclidean_matches = [[(636, 788), (912, 369)],
 [(572, 891), (906, 487)],
 [(166, 898), (565, 684)],
 [(612, 751), (874, 349)],
 [(768, 666), (966, 203)],
 [(626, 961), (985, 521)],
 [(528, 428), (650, 115)],
 [(487, 923), (849, 554)],
 [(168, 892), (563, 678)],
 [(96, 701), (412, 550)]]

euclidean_matches, euclidean_matches_test = euclidean_matches[0:2], euclidean_matches[2:]

affine_matches = [[(388, 416), (372, 408)],
 [(604, 54), (950, 227)],
 [(712, 896), (216, 648)],
 [(898, 554), (744, 477)],
 [(614, 120), (894, 260)],
 [(554, 118), (836, 259)],
 [(996, 516), (880, 458)],
 [(271, 276), (395, 338)],
 [(693, 154), (939, 277)],
 [(468, 786), (82, 593)]]

affine_matches, affine_matches_test = affine_matches[0:3], affine_matches[3:]

projection_matches = [[(778, 358), (699, 632)],
 [(575, 285), (551, 485)],
 [(637, 147), (670, 423)],
 [(888, 628), (667, 916)],
 [(409, 828), (157, 756)],
 [(355, 387), (339, 460)],
 [(653, 889), (296, 952)],
 [(879, 705), (612, 975)],
 [(534, 860), (223, 851)],
 [(553, 328), (512, 503)]]

projection_matches, projection_matches_test = projection_matches[0:4], projection_matches[4:]


In [208]:
def get_translation_matrix(point_pairs):

    a = point_pairs[0][1][0] - point_pairs[0][0][0]
    b = point_pairs[0][1][1] - point_pairs[0][0][1]

    transition_matrix = np.array([[1, 0, a],
                                   [0, 1, b],
                                   [0, 0, 1]])

    return transition_matrix

def get_euclidean_matrix(point_pairs):

    solution_matrix = []
    solution_vector = []
    for pair in point_pairs:
        x, y = pair[0]
        x_, y_ = pair[1]

        solution_matrix += [[x, -1 * y, 1, 0],
                            [y,      x, 0, 1]]
        solution_vector += [x_, y_]

    solution_matrix = np.array(solution_matrix)
    solution_vector = np.array(solution_vector)

    a, b, c, d = np.linalg.solve(solution_matrix, solution_vector)

    transition_matrix = np.array([[a, -b, c],
                                  [b,  a, d],
                                  [0,  0, 1]])
    return transition_matrix

def get_affine_matrix(point_pairs):

    solution_matrix = []
    solution_vector = []
    for pair in point_pairs:
        x, y = pair[0]
        x_, y_ = pair[1]

        solution_matrix += [[x, y, 1, 0, 0, 0],
                            [0, 0, 0, x, y, 1]]
        solution_vector += [x_, y_]

    solution_matrix = np.array(solution_matrix)
    solution_vector = np.array(solution_vector)

    a, b, c, d, e, f = np.linalg.solve(solution_matrix, solution_vector)

    transition_matrix = np.array([[a, b, c],
                                  [d, e, f],
                                  [0,  0, 1]])
    return transition_matrix

def get_projection_matrix(point_pairs):

    solution_matrix = []
    solution_vector = []
    for index, pair in enumerate(point_pairs):
        x, y = pair[0]
        x_, y_ = pair[1]

        z_x = [0, 0, 0, 0]
        z_y = [0, 0, 0, 0]
        z_1 = [0, 0, 0, 0]
        z_x[index], z_y[index], z_1[index] = -x_, -y_, -1

        solution_matrix += [[x, y, 1, 0, 0, 0, 0, 0] + z_x,
                            [0, 0, 0, x, y, 1, 0, 0] + z_y,
                            [0, 0, 0, 0, 0, 0, x, y] + z_1]
        solution_vector += [0, 0, -1]

    solution_matrix = np.array(solution_matrix)
    print(solution_matrix)
    solution_vector = np.array(solution_vector)
    print(solution_vector)

    a, b, c, d, e, f, g, h, _, _, _, _ = np.linalg.solve(solution_matrix, solution_vector)

    transition_matrix = np.array([[a, b, c],
                                  [d, e, f],
                                  [g, h, 1]])
    return transition_matrix

def get_transition_matrix(point_pairs):

    if len(point_pairs) == 1:
        return get_translation_matrix(point_pairs)
    elif len(point_pairs) == 2:
        return get_euclidean_matrix(point_pairs)
    elif len(point_pairs) == 3:
        return get_affine_matrix(point_pairs)
    elif len(point_pairs) == 4:
        return get_projection_matrix(point_pairs)
    else:
        raise Exception(f'Received {len(point_pairs)}. get_transition matrix requires 1, 2, 3, or 4 pairs of points.')

In [209]:
def convert_vector(x):

    x = list(x) + [1]
    return np.array(x)

In [210]:
@np.vectorize
def round_the_right_fucking_way_please(x):

    if x % 1 == 0.5:
        x += 0.001
    
    return np.rint(x)

def test_transition_matrix(transition_matrix, pairs):
    
    result = True
    for pair in pairs: 
        x, b = map(convert_vector, pair)

        Ax = np.matmul(transition_matrix, x)
        # Divide by z_ to get back to (x_, y_, 1) space.
        Ax = Ax / Ax[2]
        Ax = round_the_right_fucking_way_please(Ax).astype(int)
        print(Ax, b)
        result *= (Ax == b).all()

    return result

In [211]:
t = get_transition_matrix(affine_matches)
t

array([[  1. ,  -1. , 400. ],
       [  0. ,   0.5, 200. ],
       [  0. ,   0. ,   1. ]])

In [212]:
T = get_transition_matrix(projection_matches)
T

[[ 778  358    1    0    0    0    0    0 -699    0    0    0]
 [   0    0    0  778  358    1    0    0 -632    0    0    0]
 [   0    0    0    0    0    0  778  358   -1    0    0    0]
 [ 575  285    1    0    0    0    0    0    0 -551    0    0]
 [   0    0    0  575  285    1    0    0    0 -485    0    0]
 [   0    0    0    0    0    0  575  285    0   -1    0    0]
 [ 637  147    1    0    0    0    0    0    0    0 -670    0]
 [   0    0    0  637  147    1    0    0    0    0 -423    0]
 [   0    0    0    0    0    0  637  147    0    0   -1    0]
 [ 888  628    1    0    0    0    0    0    0    0    0 -667]
 [   0    0    0  888  628    1    0    0    0    0    0 -916]
 [   0    0    0    0    0    0  888  628    0    0    0   -1]]
[ 0  0 -1  0  0 -1  0  0 -1  0  0 -1]


array([[ 4.66916467e-01, -4.41243184e-01,  2.85869131e+02],
       [ 1.71383682e-01,  4.38922004e-01,  1.53615730e+02],
       [-3.27931175e-04, -1.17880810e-04,  1.00000000e+00]])

In [213]:
test_transition_matrix(T, projection_matches+projection_matches_test)

[699. 632.   1.] [699 632   1]
[699 632   1] [699 632   1]
[551. 485.   1.] [551 485   1]
[551 485   1] [551 485   1]
[670. 423.   1.] [670 423   1]
[670 423   1] [670 423   1]
[667. 916.   1.] [667 916   1]
[667 916   1] [667 916   1]
[145.11628505 764.23449395   1.        ] [157 756   1]
[145 764   1] [157 756   1]
[335.17332179 458.6348535    1.        ] [339 460   1]
[335 459   1] [339 460   1]
[291.45591418 962.80241993   1.        ] [296 952   1]
[291 963   1] [296 952   1]
[612.76837825 976.23366904   1.        ] [612 975   1]
[613 976   1] [612 975   1]
[215.24785725 860.54083765   1.        ] [223 851   1]
[215 861   1] [223 851   1]
[511.98939237 503.02920431   1.        ] [512 503   1]
[512 503   1] [512 503   1]


False

In [214]:
test_transition_matrix(t, affine_matches_test + affine_matches)

[744. 477.   1.] [744 477   1]
[744 477   1] [744 477   1]
[894. 260.   1.] [894 260   1]
[894 260   1] [894 260   1]
[836. 259.   1.] [836 259   1]
[836 259   1] [836 259   1]
[880. 458.   1.] [880 458   1]
[880 458   1] [880 458   1]
[395. 338.   1.] [395 338   1]
[395 338   1] [395 338   1]
[939. 277.   1.] [939 277   1]
[939 277   1] [939 277   1]
[ 82. 593.   1.] [ 82 593   1]
[ 82 593   1] [ 82 593   1]
[372. 408.   1.] [372 408   1]
[372 408   1] [372 408   1]
[950. 227.   1.] [950 227   1]
[950 227   1] [950 227   1]
[216. 648.   1.] [216 648   1]
[216 648   1] [216 648   1]


True

In [215]:
(np.array([400., 350., 1.]) == np.array([400, 350, 1])).all()

True

In [216]:
affine_matches_test

[[(898, 554), (744, 477)],
 [(614, 120), (894, 260)],
 [(554, 118), (836, 259)],
 [(996, 516), (880, 458)],
 [(271, 276), (395, 338)],
 [(693, 154), (939, 277)],
 [(468, 786), (82, 593)]]

In [217]:
simple_img = cv2.imread('part2-images/Simple.jpg')

In [218]:
def Apply_Transformation_Two(img, transform_array):

    # Initialize new image
    new_img = np.zeros(shape=img.shape)

    # Loop through coordinates and
    # apply inverse transformation
    for r in range(img.shape[0]):
        for c in range(img.shape[1]):
            current_coor = np.array([[c, r, 1]])
            new_coor = np.matmul(current_coor, transform_array)
            new_x = new_coor[0] / new_coor[2]
            new_y = new_coor[1] / new_coor[2]
            # Update new image if old coordinate
            # is within the image coordinate bounds
            if new_y < img.shape[0] and \
                new_x < img.shape[1] and \
                new_y > 0 and \
                new_x > 0:

                    new_img[r, c] = img[int(new_y), int(new_x)]

    return new_img

In [219]:
translation_input = np.array([
    [translation_matches[0][0][0], 0],
    [0, translation_matches[0][0][1]]
])

print('Linear Equation Array:')
print(translation_input)
print()

translation_output = np.array([
    translation_matches[0][1][0], translation_matches[0][1][1]
])

print('Equation Output:')
print(translation_output)
print()

translation_solved = np.linalg.solve(translation_input, translation_output)

print('Equation Solved:')
print(translation_solved)
print()

translation_transform = np.array([
    [translation_solved[0], 0, 0],
    [0, translation_solved[1], 0],
    [0, 0, 1]
])

print('Transfrom Matrix:')
print(translation_transform)
print()

translation_original = np.array([
    translation_matches_test[0][0][0], translation_matches_test[0][0][1], 1
])

translation_new = np.matmul(translation_original, translation_transform)

print(f'Test: ({translation_new[0] / translation_new[2]}, {translation_new[1] / translation_new[2]}) = {translation_matches_test[0][1]}')

TypeError: 'int' object is not subscriptable

In [ ]:
euclidean_input = np.array([
    [euclidean_matches[0][0][0], euclidean_matches[0][0][1], 0, 0],
    [0, 0, euclidean_matches[0][0][0], euclidean_matches[0][0][1]],
    [euclidean_matches[1][0][0], euclidean_matches[1][0][1], 0, 0],
    [0, 0, euclidean_matches[1][0][0], euclidean_matches[1][0][1]],
])

print('Linear Equation Array:')
print(euclidean_input)
print()

euclidean_output = np.array([
    euclidean_matches[0][1][0], euclidean_matches[0][1][1], euclidean_matches[1][1][0], euclidean_matches[1][1][1]
])

print('Equation Output:')
print(euclidean_output)
print()

euclidean_solved = np.linalg.solve(euclidean_input, euclidean_output)

print('Equation Solved:')
print(euclidean_solved)
print()

euclidean_transform = np.array([
    [euclidean_solved[0], euclidean_solved[1], 0],
    [euclidean_solved[2], euclidean_solved[3], 0],
    [0, 0, 1]
])

print('Transfrom Matrix:')
print(euclidean_transform)
print()

euclidean_original = np.array([
    [euclidean_matches_test[0][0][0], euclidean_matches_test[0][0][1], 1],
])

euclidean_new = np.matmul(euclidean_original[0], euclidean_transform)

print(f'Test: ({euclidean_new[0] / euclidean_new[2]}, {euclidean_new[1] / euclidean_new[2]}) = {euclidean_matches_test[0][1]}')

Linear Equation Array:
[[487 300   0   0]
 [  0   0 487 300]
 [500 319   0   0]
 [  0   0 500 319]]

Equation Output:
[555  25 575  35]

Equation Solved:
[ 0.8490566   0.47169811 -0.47169811  0.8490566 ]

Transfrom Matrix:
[[ 0.8490566   0.47169811  0.        ]
 [-0.47169811  0.8490566   0.        ]
 [ 0.          0.          1.        ]]

Test: (226.98113207547345, 459.43396226414825) = (510, 50)


In [ ]:
affine_input = np.array([
    [affine_matches[0][0][0], affine_matches[0][0][1], 1, 0, 0, 0],
    [0, 0, 0, affine_matches[0][0][0], affine_matches[0][0][1], 1],
    [affine_matches[1][0][0], affine_matches[1][0][1], 1, 0, 0, 0],
    [0, 0, 0, affine_matches[1][0][0], affine_matches[1][0][1], 1],
    [affine_matches[2][0][0], affine_matches[2][0][1], 1, 0, 0, 0],
    [0, 0, 0, affine_matches[2][0][0], affine_matches[2][0][1], 1],
])

print('Linear Equation Array:')
print(affine_input)
print()

affine_output = np.array([
    affine_matches[0][1][0], affine_matches[0][1][1], affine_matches[1][1][0], affine_matches[1][1][1], affine_matches[2][1][0], affine_matches[2][1][1]
])

print('Equation Output:')
print(affine_output)
print()

affine_solved = np.linalg.solve(affine_input, affine_output)

print('Equation Solved:')
print(affine_solved)
print()

affine_transform = np.array([
    [affine_solved[0], affine_solved[1], 0],
    [affine_solved[2], affine_solved[3], 0],
    [0, 0, 1]
])

print('Transfrom Matrix:')
print(affine_transform)
print()

affine_original = np.array([
    [affine_matches_test[0][0][0], affine_matches_test[0][0][1], 1],
])

affine_new = np.matmul(affine_original[0], affine_transform)

print(f'Test: ({affine_new[0] / affine_new[2]}, {affine_new[1] / affine_new[2]}) = {affine_matches_test[0][1]}')

Linear Equation Array:
[[300 300   1   0   0   0]
 [  0   0   0 300 300   1]
 [401 300   1   0   0   0]
 [  0   0   0 401 300   1]
 [300 302   1   0   0   0]
 [  0   0   0 300 302   1]]

Equation Output:
[400 350 501 350 398 351]

Equation Solved:
[ 1.00000000e+00 -1.00000000e+00  4.00000000e+02  1.41754162e-16
  5.00000000e-01  2.00000000e+02]

Transfrom Matrix:
[[ 1.00000000e+00 -1.00000000e+00  0.00000000e+00]
 [ 4.00000000e+02  1.41754162e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Test: (121300.00000000065, -500.0000000000037) = (598, 351)
